# 📝 Textová klasifikace a Named Entity Recognition (NER)

**Autor:** Praut s.r.o. - AI Integration & Business Automation

## Co se naučíte:
- Klasifikace textu do kategorií
- Rozpoznávání entit (jména, místa, organizace)
- Automatizace třídění dokumentů
- Extrakce klíčových informací z textu

In [ ]:
!pip install -q transformers accelerate torch datasets pandas

In [ ]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer
import torch
import pandas as pd

device = 0 if torch.cuda.is_available() else -1
print(f"🖥️ Device: {'GPU' if device == 0 else 'CPU'}")

## 1. Textová klasifikace

### 1.1 Klasifikace zpráv/ticketů

In [ ]:
# Zero-shot klasifikace - klasifikace bez specifického tréninku
classifier = pipeline("zero-shot-classification", 
                      model="facebook/bart-large-mnli",
                      device=device)

# Definice kategorií pro support tickety
kategorie = ["technický problém", "fakturace", "reklamace", "dotaz na produkt", "stížnost"]

zpravy = [
    "Dobrý den, nemůžu se přihlásit do systému, stále mi to hlásí chybu hesla.",
    "Prosím o vystavení opravné faktury, chybí tam DPH.",
    "Produkt přišel poškozený, chci vrátit peníze.",
    "Jaké jsou rozměry modelu XL?",
    "Jsem velmi nespokojen s rychlostí dodání, trvalo to 3 týdny!"
]

print("🎫 Automatická klasifikace support ticketů:\n")
for zprava in zpravy:
    result = classifier(zprava, kategorie, multi_label=False)
    print(f"📝 {zprava[:60]}...")
    print(f"   → {result['labels'][0]} ({result['scores'][0]:.1%})\n")

In [ ]:
# Multi-label klasifikace - více kategorií najednou
tagy = ["urgent", "VIP zákazník", "eskalace", "automatizovatelné"]

zprava = "URGENTNÍ: Náš enterprise systém je down, potřebujeme okamžitou pomoc! Jsme váš největší klient."

result = classifier(zprava, tagy, multi_label=True)

print(f"📝 Zpráva: {zprava}\n")
print("🏷️ Přiřazené tagy:")
for label, score in zip(result['labels'], result['scores']):
    if score > 0.5:  # Práh 50%
        print(f"   ✅ {label}: {score:.1%}")
    else:
        print(f"   ❌ {label}: {score:.1%}")

### 1.2 Klasifikace e-mailů

In [ ]:
# Simulace příchozích e-mailů
emaily = pd.DataFrame({
    "od": ["jan@firma.cz", "partner@company.com", "spam@xxx.com", "hr@korporace.cz"],
    "predmet": [
        "Nabídka spolupráce na novém projektu",
        "Invoice #12345 - Payment reminder",
        "You won $1,000,000!!!",
        "Pozvánka na pohovor - Senior Developer"
    ],
    "obsah": [
        "Dobrý den, rádi bychom s vámi probrali možnosti spolupráce na AI projektu.",
        "Dear partner, please find attached invoice for services rendered in Q4.",
        "Click here to claim your prize! Limited time offer! Act now!",
        "Vážený pane, zveme Vás na osobní pohovor na pozici Senior Developer."
    ]
})

email_kategorie = ["obchodní", "fakturace", "spam", "HR/kariéra", "osobní"]

# Klasifikace všech e-mailů
vysledky = []
for _, row in emaily.iterrows():
    text = f"{row['predmet']}. {row['obsah']}"
    result = classifier(text, email_kategorie)
    vysledky.append({
        "kategorie": result['labels'][0],
        "jistota": result['scores'][0]
    })

emaily["kategorie"] = [v["kategorie"] for v in vysledky]
emaily["jistota"] = [v["jistota"] for v in vysledky]

print("📧 Automaticky roztříděné e-maily:")
print(emaily[["od", "predmet", "kategorie", "jistota"]].to_string(index=False))

## 2. Named Entity Recognition (NER)

NER rozpoznává a extrahuje entity jako:
- **PER** - Osoby
- **ORG** - Organizace
- **LOC** - Místa
- **MISC** - Ostatní (data, produkty...)

In [ ]:
# NER pipeline
ner = pipeline("ner", 
               model="dbmdz/bert-large-cased-finetuned-conll03-english",
               aggregation_strategy="simple",
               device=device)

text = """Apple Inc. CEO Tim Cook announced a new partnership with Microsoft 
at a press conference in San Francisco. The deal, worth $500 million, 
will focus on artificial intelligence research."""

entities = ner(text)

print(f"📝 Text: {text}\n")
print("🏷️ Nalezené entity:")
for entity in entities:
    emoji = {"ORG": "🏢", "PER": "👤", "LOC": "📍", "MISC": "📦"}.get(entity['entity_group'], "❓")
    print(f"   {emoji} {entity['word']} ({entity['entity_group']}) - {entity['score']:.1%}")

In [ ]:
# Vícejazyčný NER model
ner_multilang = pipeline("ner", 
                         model="Babelscape/wikineural-multilingual-ner",
                         aggregation_strategy="simple",
                         device=device)

cesky_text = """Společnost Škoda Auto sídlící v Mladé Boleslavi oznámila spolupráci 
s firmou Praut s.r.o. z Chebu. Generální ředitel Thomas Schäfer potvrdil investici 
ve výši 2 miliardy korun do vývoje elektromobilů."""

entities = ner_multilang(cesky_text)

print(f"📝 Český text: {cesky_text}\n")
print("🏷️ Nalezené entity:")
for entity in entities:
    emoji = {"ORG": "🏢", "PER": "👤", "LOC": "📍", "MISC": "📦"}.get(entity['entity_group'], "❓")
    print(f"   {emoji} {entity['word']} ({entity['entity_group']}) - {entity['score']:.1%}")

## 3. Praktická automatizace: Extrakce kontaktů z textu

In [ ]:
import re

def extrahuj_kontakty(text):
    """Extrahuje kontaktní informace z textu pomocí NER a regex."""
    
    # NER pro jména a organizace
    entities = ner_multilang(text)
    
    kontakt = {
        "osoby": [],
        "organizace": [],
        "mista": [],
        "emaily": [],
        "telefony": []
    }
    
    for entity in entities:
        if entity['entity_group'] == 'PER':
            kontakt['osoby'].append(entity['word'])
        elif entity['entity_group'] == 'ORG':
            kontakt['organizace'].append(entity['word'])
        elif entity['entity_group'] == 'LOC':
            kontakt['mista'].append(entity['word'])
    
    # Regex pro e-maily a telefony
    kontakt['emaily'] = re.findall(r'[\w.+-]+@[\w-]+\.[\w.-]+', text)
    kontakt['telefony'] = re.findall(r'\+?\d[\d\s-]{8,}\d', text)
    
    # Odstranění duplikátů
    for key in kontakt:
        kontakt[key] = list(set(kontakt[key]))
    
    return kontakt

# Test na vizitce
vizitka = """
Jan Novák
Senior Developer
Praut s.r.o.
Cheb, Česká republika
Email: jan.novak@praut.cz
Tel: +420 123 456 789
"""

kontakt = extrahuj_kontakty(vizitka)

print("📇 Extrahované kontaktní údaje:")
print(f"   👤 Osoby: {', '.join(kontakt['osoby']) or 'nenalezeno'}")
print(f"   🏢 Organizace: {', '.join(kontakt['organizace']) or 'nenalezeno'}")
print(f"   📍 Místa: {', '.join(kontakt['mista']) or 'nenalezeno'}")
print(f"   📧 E-maily: {', '.join(kontakt['emaily']) or 'nenalezeno'}")
print(f"   📞 Telefony: {', '.join(kontakt['telefony']) or 'nenalezeno'}")

## 4. Automatizace: Zpracování faktur

In [ ]:
def zpracuj_fakturu(text):
    """Extrahuje klíčové informace z textu faktury."""
    
    entities = ner_multilang(text)
    
    info = {
        "dodavatel": None,
        "odberatel": None,
        "castky": [],
        "data": []
    }
    
    orgs = [e['word'] for e in entities if e['entity_group'] == 'ORG']
    if len(orgs) >= 2:
        info['dodavatel'] = orgs[0]
        info['odberatel'] = orgs[1]
    elif len(orgs) == 1:
        info['dodavatel'] = orgs[0]
    
    # Extrakce částek
    info['castky'] = re.findall(r'[\d\s]+[.,]?\d*\s*(?:Kč|CZK|EUR|USD|€|\$)', text, re.IGNORECASE)
    
    # Extrakce dat
    info['data'] = re.findall(r'\d{1,2}[./]\d{1,2}[./]\d{2,4}', text)
    
    return info

faktura_text = """
FAKTURA č. 2024-001

Dodavatel: Praut s.r.o., IČO: 12345678
Odběratel: ABC Corporation a.s.

Datum vystavení: 15.01.2024
Datum splatnosti: 29.01.2024

Položky:
- AI konzultace: 50 000 Kč
- Vývoj automatizace: 120 000 Kč

Celkem bez DPH: 170 000 Kč
DPH 21%: 35 700 Kč
CELKEM: 205 700 Kč
"""

info = zpracuj_fakturu(faktura_text)

print("📄 Extrahované informace z faktury:")
print(f"   🏢 Dodavatel: {info['dodavatel']}")
print(f"   🏢 Odběratel: {info['odberatel']}")
print(f"   📅 Data: {', '.join(info['data'])}")
print(f"   💰 Částky: {', '.join(info['castky'])}")

## 5. Batch zpracování dokumentů

In [ ]:
# Simulace dávky dokumentů
dokumenty = [
    {"id": 1, "text": "Microsoft CEO Satya Nadella visited Prague for a tech conference."},
    {"id": 2, "text": "Tesla announced new factory plans in Berlin, Germany."},
    {"id": 3, "text": "Czech company Avast was acquired by NortonLifeLock for $8 billion."},
]

# Hromadné zpracování
vysledky = []
for doc in dokumenty:
    entities = ner(doc['text'])
    
    # Kategorizace dokumentu podle obsahu
    org_count = len([e for e in entities if e['entity_group'] == 'ORG'])
    loc_count = len([e for e in entities if e['entity_group'] == 'LOC'])
    per_count = len([e for e in entities if e['entity_group'] == 'PER'])
    
    vysledky.append({
        "id": doc['id'],
        "organizace": org_count,
        "mista": loc_count,
        "osoby": per_count,
        "entity_list": [e['word'] for e in entities]
    })

df = pd.DataFrame(vysledky)
print("📊 Přehled entit v dokumentech:")
print(df.to_string(index=False))

---
## 🏁 Shrnutí

- ✅ Zero-shot klasifikace pro flexibilní třídění bez tréninku
- ✅ NER pro extrakci entit (osoby, firmy, místa)
- ✅ Automatizace zpracování e-mailů a dokumentů
- ✅ Extrakce strukturovaných dat z nestrukturovaného textu

**Další notebook:** Sentiment analýza a analýza emocí